In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:12pt; line-height:40px;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 </span>

In [5]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama

# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

# <span style="color:rgb(0,200,100);">3. 문서를 chunk로 분할하기 </span>

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법_with_markdown.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)
# documents = loader.load_and_split(text_splitter)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 청크 생성")

총 194개 청크 생성


# <span style="color:rgb(0,200,100);">4. metadata 추가하기 </span>
- 청크 내용의 카테고리, 청크내용의 title, 조항

In [3]:
import re
def remove_special_chars(text:str) -> str:
    "특수문자 및 \n제거(:는 그대로)"
    # \n제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,\.:]', '', text)
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned
# 사용예시
content = "**소득세 납세 의무 **: \n\n이 법 또는 「법인세법」에 따라 소득에 대한 소득세를 부과"
print(remove_special_chars(content))

소득세 납세 의무 : 이 법 또는 법인세법에 따라 소득에 대한 소득세를 부과


In [4]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
    )
    prompt = f"""다음 소득세법 조문의 핵심 제목을 100토큰 이내로 완벽하게 말이 되도록 추출해 주세요.
    중간에 말이 끊기면 안 되요.
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 종합과세 분리과세 합산과세 종합소득과세표준 분리과세소득 금융투자소득
"""
print(extract_title_with_llm(content))

소득세 납세 범위: 원천징수 대상의 거주지 기반 과세 거주자: 주된 사업장 소재지, 비거주자: 주된 국내 사업장 소재지


In [5]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_openai import ChatOpenAI
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOpenAI(
        model = OPENAI_LLM_MODEL,
        temperature = 0.1
    )
    prompt = f"""다음 소득세법 조문의 핵심 제목을 100토큰 이내로 완벽하게 말이 되도록 추출해 주세요.
    중간에 말이 끊기면 안 되요.
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 종합과세 분리과세 합산과세 종합소득과세표준 분리과세소득 금융투자소득
"""
print(extract_title_with_llm(content))

원천징수자의 거주지 및 사업장 소재지에 따른 소득세 과세 기준


In [8]:
def categorize_content(content):
    "내용 카테고리 분류"
    # 쳇 gpt가 meta 데이터에 쓴다고 하니 만들어 준 카테고리
    categories = {
    '납세의무': ['납세의무', '거주자', '비거주자', '원천징수', '공동사업자', '상속인', '증여자', '신탁재산'],
    '세율계산': ['세율', '과세표준', '산출세액', '결정세액', '종합소득', '퇴직소득'],
    '근로소득': ['근로소득', '총급여', '급여', '연봉', '임금', '퇴직소득'],
    '사업소득': ['사업소득', '공동사업', '주택임대소득', '부업소득'],
    '이자배당': ['이자소득', '배당소득', '예금이자', '채권', '의제배당'],
    '양도소득': ['양도소득', '자산양도', '부동산양도', '주식양도'],
    '연금소득': ['연금소득', '공적연금', '사적연금', '연금보험'],
    '기타소득': ['기타소득', '상금', '보상금', '발명보상금', '종교인소득'],
    '공제감면': ['공제', '소득공제', '세액공제', '기본공제', '감면'],
    '비과세': ['비과세', '면제', '복무급여', '실업급여', '출산휴가급여', '장학금'],
    '신고납부': ['신고', '납부', '납세지', '신고기한', '원천징수'],
    '과세기간': ['과세기간', '과세연도', '사업연도'],
    '과세소득구분': ['종합소득', '퇴직소득', '양도소득', '금융투자소득'],
    }
    result = [] # 각 카테고리 추가
    for category, keywords in categories.items():
        #print(category, keywords)
        #print(any([keyword in content for keyword in keywords]))
        if any([keyword in content for keyword in keywords]):
            result.append(category)
    if not result:
        result.append("기타")
    return result
categorize_content(content)

['납세의무', '세율계산', '신고납부', '과세소득구분']

- 개선된 카테고리 분류방법 : 키워드의 중요도에 따라 가중치를 부여하고, 가중치가 높은 순으로 카테고리 반환

In [54]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },

        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },

        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },

        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },

        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },

        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },

        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },

        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },

        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },

        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },

        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },

        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },

        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },

        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [55]:
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 종합과세 분리과세 합산과세 종합소득과세표준 분리과세소득 금융투자소득
"""

In [59]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            if keyword in content:
                score += weight
            #count = content.count(keyword) 나온 횟수로 
            #score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    # print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories
categorize_content(content)

['과세방식', '납세의무', '세율계산']

In [12]:
if __name__ == "__main__":
    print(categorize_content("연봉 5000만원인 회사원의 소득세는 얼마예요?"))

['세율계산', '근로소득']


In [153]:
categorize_content(documents[46].page_content)

['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간']

In [9]:
import re
article_match = re.findall(r'제(\d+)조', "제60조 뭐라고 제60조 무시기 제9조 제70조")
article = list(set(article_match))
article.sort(key = lambda x : int(x))
[ f"{a}조" for a in article ]
print("「"+ ", ".join([f"{a}조" for a in article]) +"」")

「9조, 60조, 70조」


In [10]:
# 해당 조항 추출하기
import re
def get_article(content):
    "조항들 추출"
    article_match = re.findall(r'제(\d+)조', content)
    article = list(set(article_match)) # 중복제거
    article.sort(key=lambda x:int(x))
    if article:
        return "「"+ ", ".join([f"{a}조" for a in article]) +"」"
    else:
        return ""
print(get_article("무시기 무시기"))

In [1]:
#documents

In [26]:
%%time
# Exaone으로 메타데이터 추가
enhanced_chunks = []# 메타데이터(source, title, category, article)를 추가할 청크 리스트
print("Exaone으로 제목 추출 중...")
for i, chunk in enumerate(documents):
    if i % 20 == 0:
        print(f"   진행: {i/len(documents)*100:.1f}% 진행")
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    # Exaone으로 제목 추출
    metadata['title'] = extract_title_with_llm(content) # llm으로 제목 뽑기
    metadata['category'] = categorize_content(content) # 카테고리 뽑기
    metadata['chunk_id'] = f"chunk_{i:03d}"
    # 조문 번호 모두 추출
    article = get_article(content)
    if article:
        metadata['article'] = article
    enhanced_chunks.append(type(chunk)(
        page_content=content,
        metadata=metadata
    ))
print(f"✅ {len(enhanced_chunks)}개 청크 처리 완료")

Exaone으로 제목 추출 중...
   진행: 0.0% 진행
   진행: 10.3% 진행
   진행: 20.6% 진행
   진행: 30.9% 진행
   진행: 41.2% 진행
   진행: 51.5% 진행
   진행: 61.9% 진행
   진행: 72.2% 진행
   진행: 82.5% 진행
   진행: 92.8% 진행
✅ 194개 청크 처리 완료
CPU times: total: 2.98 s
Wall time: 4min 50s


In [28]:
type(enhanced_chunks[0].metadata['category'])

list

In [29]:
import pandas as pd

# enhanced_chunks를 DataFrame으로 변환
data = []
for chunk in enhanced_chunks:
    row = {"page_content":chunk.page_content}
    row.update(chunk.metadata)
    data.append(row)
df = pd.DataFrame(data)

# CSV 저장
df.to_csv("./tax_docs/소득세법enhanced_chunks.csv", index=False, encoding="cp949")

## <span style="color:rgb(0,200,100);">title 추출에 시간이 너무 오래 걸릴 경우</span>

In [30]:
import pandas as pd
df = pd.read_csv('./tax_docs/소득세법enhanced_chunks.csv', encoding="cp949")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   page_content  194 non-null    object
 1   source        194 non-null    object
 2   title         194 non-null    object
 3   category      194 non-null    object
 4   chunk_id      194 non-null    object
 5   article       193 non-null    object
dtypes: object(6)
memory usage: 9.2+ KB


In [31]:
type(df.loc[0,'category'])

str

In [32]:
import ast

# 입력 문자열
input_str = "['납세의무', '신고납부']"

# 리스트로 변환
result_list = ast.literal_eval(input_str)

print(result_list)

['납세의무', '신고납부']


In [33]:
# 'category' 컬럼의 문자열을 리스트 객체로 변환
df['category'] = df['category'].apply(ast.literal_eval)
type(df.loc[0,'category'])

list

In [34]:
for i, row in df.loc[0:2].iterrows():
    print(i, row['category'])

0 ['납세의무', '이자배당', '사업소득', '세율계산', '근로소득', '양도소득', '기타소득', '신고납부']
1 ['납세의무', '세율계산', '양도소득', '사업소득', '과세방식', '신고납부']
2 ['납세의무', '이자배당', '양도소득', '근로소득', '사업소득', '연금소득', '기타소득', '신고납부', '과세기간']


In [35]:
%%time
from langchain_core.documents import Document
import numpy as np
enhanced_chunks = []

for i, row in df.iterrows():
    if i%100==0:
        print(type(row.get("category")))
    content = row["page_content"],
    metadata = metadata = {
        "source": row.get("source"),
        "title": row.get("title"),
        "category": row.get("category"),
        "chunk_id": row.get("chunk_id"),
    }
    # article이 결측치가 아니면 추가
    if pd.notna(row.get("article")):
        metadata["article"] = row.get("article")
    # Document 객체 생성 및 리스트 추가
    doc = Document(
        page_content=row["page_content"],
        metadata=metadata
    )
    enhanced_chunks.append(doc)
print(f"✅ {len(enhanced_chunks)}개 청크 처리 완료")

<class 'list'>
<class 'list'>
✅ 194개 청크 처리 완료
CPU times: total: 31.2 ms
Wall time: 39.6 ms


# <span style="color:rgb(0,200,100);">5. 임베딩 모델 설정 </span>

In [36]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    # openai_api_key=OPENAI_API_KEY
)

In [37]:
print(type(enhanced_chunks[46].metadata['category']))

<class 'list'>


# <span style="color:rgb(0,200,100);">6. Pinecone 인덱스 생성 및 vector store(DB) 저장 </span>

In [38]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import time
# pinecone 클라언트
pc = Pinecone(api_key=PINECONE_API_KEY)
#print("pinecone index들 :", pc.list_indexes().names())
# 인덱스 생성 여부 확인 및 생성
#if PINECONE_INDEX_NAME not in pc.list_indexes().names():
if not pc.has_index(PINECONE_INDEX_NAME):
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DEMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION,
                           cloud=PINECONE_INDEX_CLOUD)
    )
    print('인덱스 생성 완료')
else:
    PineconeVectorStore.from_documents(
               documents=[enhanced_chunks[0]],
               embedding=embedding,
               index_name=PINECONE_INDEX_NAME
           )
    index = pc.Index(PINECONE_INDEX_NAME)
    index.delete(delete_all=True) # index안의 데이터 삭제
    ##pc.delete_index(PINECONE_INDEX_NAME) # 인덱스 자체를 지우고 싶으면 이렇게 함
    print(f"인덱스 '{PINECONE_INDEX_NAME}'가 이미 존재하여 모든 데이터를 삭제하였습니다")
    #print(f"인덱스 {PINECONE_INDEX_NAME}이 이미 존재합니다")

인덱스 'better-rag-index'가 이미 존재하여 모든 데이터를 삭제하였습니다


In [39]:
# 빈 인덱스가 잘 생성되었는지 확인
index = pc.Index(PINECONE_INDEX_NAME)
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [40]:
%%time
# Pinecone 벡터 스토어 업로드
vector_database = PineconeVectorStore.from_documents(
    documents=enhanced_chunks,
    embedding=embedding,
    index_name = PINECONE_INDEX_NAME
)
print("벡터 DB 저장 완료")

#업로드한 벡터db를 가져올 때
# vector_database = PineconeVectorStore(
#     embedding=embedding,# 질문을 임베딩하여 유사도 검색
#     index_name=PINECONE_INDEX_NAME
# )

벡터 DB 저장 완료
CPU times: total: 8.34 s
Wall time: 16.5 s


# <span style="color:rgb(0,200,100);">7. 관련문서 미리보기(metadata활용) </span>

In [41]:
query = "연봉 5000만원인 직장인의 소득세는 얼마예요?"
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 얼마예요?"
categorize_content(query)

['납세의무', '세율계산', '근로소득']

In [42]:
# Retriever 생성

# 정확하게 ['근로소득','세율계산']인 경우
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":4,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
docs = retriever.invoke(query)
print("관련 문서")
for i, doc in enumerate(docs):
    chunk_id = doc.metadata.get('chunk_id')
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}:{article} {category} - {title}")

관련 문서
1.chunk_046:「14조, 17조, 55조, 56조」 ['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간'] - 제55조세율 거주자의 종합소득에 대한 소득세 세율 및 퇴직소득 산출세액 계산 방법.
2.chunk_038:「4조, 16조, 45조, 46조, 47조, 133조, 156조」 ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
3.chunk_005:「6조, 9조, 10조, 11조, 12조, 99조」 ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] - 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.
4.chunk_002:「2조, 3조, 4조, 5조, 9조, 17조, 100조, 119조, 251조」 ['납세의무', '이자배당', '양도소득', '근로소득', '사업소득', '연금소득', '기타소득', '신고납부', '과세기간'] - 공동 소유 자산의 양도소득금액 계산 및 납세의무


In [26]:
[{"category": {"$eq": cat}} for cat in categorize_content(query)]

[{'category': {'$eq': '납세의무'}},
 {'category': {'$eq': '세율계산'}},
 {'category': {'$eq': '근로소득'}}]

In [43]:
# Retriever 생성

# $or를 사용하여 리스트 내에 하나라도 포함된 문서 찾기
filter_condition = {
    "$and":[{"category": {"$eq": cat}} for cat in categorize_content(query)]
}
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":4,
        "filter":filter_condition
    }
)
docs = retriever.invoke(query)
print("관련 문서")
for i, doc in enumerate(docs):
    chunk_id = doc.metadata.get('chunk_id')
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}:{article} {category} - {title}")

관련 문서
1.chunk_038:「4조, 16조, 45조, 46조, 47조, 133조, 156조」 ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
2.chunk_005:「6조, 9조, 10조, 11조, 12조, 99조」 ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] - 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.
3.chunk_051:「56조, 57조, 58조, 59조」 ['근로소득', '세율계산', '공제감면', '사업소득', '납세의무', '과세기간'] - 제58조재해손실세액공제 및 제59조근로소득세액공제
4.chunk_009:「12조, 13조, 14조, 16조, 17조, 19조, 20조, 21조, 22조, 24조, 26조, 27조, 29조, 31조, 35조, 37조, 39조, 41조, 46조, 47조, 50조, 51조, 52조, 129조」 ['세율계산', '근로소득', '이자배당', '공제감면', '납세의무', '사업소득', '기타소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제14조: 과세표준의 계산 및 종합소득과세표준의 정의, 소득의 합산 제외 항목.


# <span style="color:rgb(0,200,100);">8. langchain 답변 생성 </span>

In [47]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

In [44]:
# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model = OPENAI_LLM_MODEL)
embedding = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)
# 2. # 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=PINECONE_INDEX_NAME
)

In [48]:
# 3. 키워드 사전 활용
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,
                api_key=OPENAI_API_KEY)
keyword_chain = prompt | llm | StrOutputParser()
query = "연봉 5000만원인 회사원의 소득세는 얼마예요?"
keyword_chain.invoke({"question":query})

'종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

In [49]:
keyword_chain.invoke({"question":"월급이 높은 남자가 있습니다"})

'근로소득이 있는 거주자가 높은 소득을 가지고 있습니다.'

In [50]:
# 3. Retriever 생성
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
final_chain = keyword_chain | rag_chain
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
result = final_chain.invoke(query)
print(result + "\n*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*")

종합소득 5천만원인 근로소득이 있는 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 5천만원에서 1,400만원을 초과하는 금액은 3,600만원이므로, 3,600만원의 15%는 540만원입니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.
*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*


# <span style="color:rgb(0,200,100);">9. langchain 답변 생성 (참조 조항 포함) </span>

In [80]:
# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model = OPENAI_LLM_MODEL)
embedding = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

# 2. 업로드한 벡터db를 가져올 때
vector_database = PineconeVectorStore(
    embedding=embedding,  # 질문을 임베딩하여 유사도 검색
    index_name=PINECONE_INDEX_NAME
)
# 3. 키워드 사전 활용
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,
                api_key=OPENAI_API_KEY)
keyword_chain = prompt | llm | StrOutputParser()
query = "연봉 5000만원인 회사원의 소득세는 얼마예요?"
keyword_chain.invoke({"question":query})

'종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

In [77]:
# 4. get_category함수와  categorize_content 함수 (기존 코드 그대로)

In [81]:
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 얼마예요?"
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
documents = retriever.invoke(query)

In [83]:
# 5. 참조 조항을 추출하는 함수
def extract_articles_from_docs(documents):
    """
    검색된 문서들에서 article 정보를 추출하여 리스트로 반환
    
    Args:
        documents: retriever에서 반환된 Document 객체 리스트
    
    Returns:
        list: 중복 제거된 조항 리스트 (예: ['제4조', '제16조', '제45조'])
    """
    articles = []
    
    for doc in documents:
        # metadata에서 article 정보 추출 (없으면 기본값)
        article = doc.metadata.get('article', '조항없음')
        
        # article이 문자열이면 그대로 사용, 아니면 처리
        if isinstance(article, str) and article != '조항없음':
            # 「4조, 16조, 45조」 형태를 분리
            article = article.replace('「', '').replace('」', '')
            article_list = [a.strip() for a in article.split(',')]
            articles.extend(article_list)
    
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    # '조' 앞에 '제'를 붙여서 표준 형식으로 변환
    # unique_articles = ['조항없음']
    final_articles = ["제"+article for article in unique_articles if article!='조항없음']
    if final_articles:
        return "소득세법 " + ", ".join(final_articles)
    else:
        return unique_articles[0]
extract_articles_from_docs(documents)

'소득세법 제4조, 제6조, 제9조, 제10조, 제11조, 제12조, 제14조, 제16조, 제17조, 제45조, 제46조, 제47조, 제55조, 제56조, 제99조, 제133조, 제156조'

In [84]:
# 6. 검색된 document를 텍스트로 변환하는 함수 (기존)
def format_documents(documents):
    """Document 리스트를 하나의 텍스트로 변환"""
    return "\n\n---\n\n".join([doc.page_content for doc in documents])

In [99]:
# 7. RAG 체인 구성 - 참조 조항 정보를 포함한 버전
# 7-1. 질문을 표준화
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?" #출력함
normalized_query = keyword_chain.invoke({"question": query}) #출력함
# 3. Retriever 생성
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)

# 7-2. 관련 문서 검색
retrieved_docs = retriever.invoke(normalized_query)
# 7-3. 참조 조항 추출
referenced_articles = extract_articles_from_docs(retrieved_docs) #출력함

# 7-4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)


# 7-5 RAG 체인 구성 - 참조 조항 정보를 포함한 버전
prompt_chain = prompt | llm | StrOutputParser()
result = prompt_chain.invoke({
            "context":format_documents(retrieved_docs), 
            "query":query
})

In [101]:
print(f"원본 질문: {query}")
print(f"표준화된 질문: {normalized_query}\n")
print("\n" + "="*50)
print("답변:")
print("="*50)
print(result)
print("\n" + "-"*50)
print(f"참조: {referenced_articles}")
print("-"*50)
print("\n*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*")

원본 질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
표준화된 질문: 연봉 5천만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가요?


답변:
연봉 5천만원인 직장인의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 즉, 5천만원에서 1,400만원을 제외한 3,600만원에 대해 15%를 적용하면 540만원이 추가됩니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.

--------------------------------------------------
참조: 소득세법 제4조, 제6조, 제9조, 제10조, 제11조, 제12조, 제14조, 제16조, 제17조, 제45조, 제46조, 제47조, 제55조, 제56조, 제99조, 제133조, 제156조
--------------------------------------------------

*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*


In [107]:
# 8. 재사용 가능한 함수로 만들기
def ask_with_reference(query: str, k: int = 3):
    """
    질문에 답변하고 참조 조항을 함께 반환하는 함수
    
    Args:
        question: 사용자 질문
        k: 검색할 문서 개수 (기본값 3)
    query, 표준화된 query, 생성된 답변, 참조조항 출력
    """
    # 질문 표준화
    normalized_query = keyword_chain.invoke({"question": query})
    # Retriever 생성
    filter_condition = {"category": {"$in": categorize_content(normalized_query)}}
    retriever = vector_database.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k, "filter": filter_condition}
    )
    # 문서 검색
    retrieved_docs = retriever.invoke(normalized_query)
    
    # 참조 조항 추출
    referenced_articles = extract_articles_from_docs(retrieved_docs)
    # 7-4. 프롬프트 템플릿
    template = f"""당신은 최고의 한국 소득세 전문가입니다.
    다음 문맥을 참고하여 질문에 답하세요.
    답을 모르면 모른다고 답하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{query}}
    문맥 : {{context}}
    답변 : """
    prompt = ChatPromptTemplate.from_template(template)


    # 7-5 RAG 체인 구성 - 참조 조항 정보를 포함한 버전
    prompt_chain = prompt | llm | StrOutputParser()
    answer = prompt_chain.invoke({
                "context":format_documents(retrieved_docs), 
                "query":query
    })
    print("\n" + "="*60)
    print(f"질문: {query}")
    print(f"(표준화: {normalized_query})")
    print("="*60)
    print("\n☑️ 답변:", answer)
    print("\n📌 참조:", referenced_articles)
    print("\n* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *\n")

In [108]:
# 12. 사용 예시

# 예시 1
ask_with_reference("연봉 5천만원인 직장인의 소득세는 얼마인가요?")


질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
(표준화: 종합소득 5천만원인 근로소득이 있는 거주자의 세액은 얼마인가요?)

☑️ 답변: 연봉 5천만원인 직장인의 소득세는 약 624만원 + (5천만원 초과 금액의 24%)로 계산됩니다. 5천만원의 경우, 624만원이 기본 세액이며, 추가 세액은 없습니다. 따라서 총 소득세는 624만원입니다.

📌 참조: 소득세법 제4조, 제14조, 제16조, 제17조, 제22조, 제45조, 제46조, 제47조, 제48조, 제49조, 제55조, 제56조, 제133조, 제156조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *



In [110]:
# 예시 2
ask_with_reference("사업소득이 있는 경우 필요경비는 어떻게 계산하나요?")

# 예시 3
ask_with_reference("양도소득세율은 얼마인가요?")


질문: 사업소득이 있는 경우 필요경비는 어떻게 계산하나요?
(표준화: 사업소득이 있는 경우 필요경비는 어떻게 계산하나요?)

☑️ 답변: 사업소득의 필요경비는 해당 과세기간의 총수입금액에 대응하는 통상적인 비용의 합계액으로 계산됩니다. 이전 과세기간에 확정된 비용 중 필요경비로 계상하지 않은 금액만 해당 과세기간의 필요경비로 인정됩니다. 필요경비의 구체적인 계산 방법은 대통령령으로 정해집니다.

📌 참조: 소득세법 제27조, 제28조, 제29조, 제30조, 제31조, 제32조, 제33조, 제52조, 제57조, 제62조, 제106조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *


질문: 양도소득세율은 얼마인가요?
(표준화: 양도소득세율은 얼마인가요?)

☑️ 답변: 양도소득세율은 자산의 종류와 보유 기간에 따라 다릅니다. 일반적으로 1년 이상 2년 미만 보유한 자산은 양도소득 과세표준의 40%가 적용되며, 2년 이상 보유한 자산은 60%가 적용됩니다. 구체적인 세율은 자산의 종류에 따라 달라질 수 있습니다.

📌 참조: 소득세법 제5조, 제55조, 제57조, 제94조, 제95조, 제103조, 제104조, 제113조, 제118조, 제127조, 제142조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *

